# 03: Velocity inversion

# NOTE: still need to organize this code more

In [1]:
#| default_exp invert

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev 
from nbdev import nbdev_export

In [3]:
#| export
import numpy as np
import pyproj
import matplotlib.path as path
import s3fs
import zarr
import matplotlib.pyplot as plt
import scipy
from datetime import timedelta
from tqdm import tqdm
import xarray as xr
import re
import pandas as pd
import geopandas as gpd
import matplotlib.path as mplp
import ipyleaflet as ipyl
from ipyleaflet import WMSLayer
import ipywidgets as ipyw
import json
import pandas as pd
from ipyleaflet import Map, WMSLayer, basemaps, GeoData
from ipywidgets import HTML
from owslib.wms import WebMapService

In [4]:
#| export
from ITS_LIVE_TOOL import setup, interactive

In [5]:
#| export
urls = []

In [10]:
#| export
data_map = interactive.Widget()

In [11]:
data_map.display()

In [39]:
#|export
try: 
    coords, gpdf, urls = interactive.return_clicked_info(data_map)
except:
    pass

1


In [ ]:
##### EMMA IN NOTEBOOK 2 ADD 'GeoData' to the import line: from ipyleaflet import Map, WMSLayer, basemaps, GeoData

t = []
wms_url = "https://glims.org/geoserver/ows?SERVICE=WMS&"
wms_layer = WMSLayer(
    url=wms_url,
    layers='GLIMS:RGI',
    transparent=True,
    format='image/png'
)


# Create a Pandas DataFrame with X and Y coordinates
#data = pd.DataFrame({'X': list(fx_map[::50]), 'Y': list(fy_map[::50])})

# Map and label widgets
map = ipyl.Map(basemap=basemaps.Esri.WorldImagery, center=(0, 0), zoom=2)
label = ipyw.Label(layout=ipyw.Layout(width="100%"))

# Create a list to store clicked URLs
urls = []

# geojson layer with hover handler
with open("catalog_v02.json") as f:
    geojson_data = json.load(f)

for feature in geojson_data["features"]:
    feature["properties"]["style"] = {
        "color": "grey",
        "weight": 1,
        "fillColor": "grey",
        "fillOpacity": 0.5,
    }

geojson_layer = ipyl.GeoJSON(data=geojson_data, hover_style={"fillColor": "red"}, selected_style = {"fillColor": "orange"})

def hover_handler(event=None, feature=None, id=None, properties=None):
    label.value = properties["zarr_url"]

def json_handler(event=None, feature=None, id=None, properties=None):    
    zarr_url = properties.get("zarr_url", "N/A")
    urls.append(zarr_url)
    print(f"Clicked URL: {zarr_url}")
    print("All Clicked URLs:", urls)
    '''
    # Plot in orange the polygon clicked on
    json_shape = [{'type': properties['geometry_epsg']['type'], 'coordinates': properties['geometry_epsg']['coordinates']}]
    geom = [shape(i) for i in json_shape]
    json_shape = gpd.GeoDataFrame({'geometry':geom})  
    json_shape = GeoData(geo_dataframe = json_shape,
                style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                name = 'Cube')

    map.add_layer(json_shape)
    '''

    
    
wms = WebMapService(wms_url)

# Initialize a list to store the geopandas dataframes
gdf_list = []

# Create a function to handle click events
def click_handler(properties=None, **kwargs):


    if kwargs.get('type') == 'contextmenu':
        latlon = kwargs.get('coordinates')
        lat, lon = latlon[0], latlon[1]
        print(f"Clicked at (Lat: {lat}, Lon: {lon})")
        
        # Arrange the coordinates
        
        response = wms.getfeatureinfo(
            layers=['GLIMS:RGI'],
            srs='EPSG:4326',
            bbox=(lon-0.001,lat-0.001,lon+0.001,lat+0.001),
            size=(1,1),
            format='image/jpeg',
            query_layers=['GLIMS:RGI'],
            info_format="application/json",
            xy=(0,0))
        df = gpd.read_file(response)
        gdf_list.append(df)  
        try:
            print(f"You have selected the glacier {df['NAME'].values[0]}, ID: {df['id'].values[0]} ")
        except:
            print(f"This glacier is not recognized by the RGI (maybe an ice-shelf ?) -> Choose another one")
        geo_data = GeoData(geo_dataframe = df,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                   name = 'Glacier')

        gdf_list.append(df) 
        map.add_layer(geo_data)

        


geojson_layer.on_hover(hover_handler)
geojson_layer.on_click(json_handler)


map.add(geojson_layer)

# Add points from the DataFrame to the map
#for index, row in data.iterrows():
#    marker = ipyl.Marker(locatfairbanks ash situationion=(row['Y'], row['X']))
#    map.add_layer(marker)

# Enable zoom when scrolling with the mouse
map.scroll_wheel_zoom = True
ipyw.VBox([map, label])


# Add the WMS layer to the map
map.add_layer(wms_layer)

# Add the click event handler to the map
map.on_interaction(click_handler)

map


In [ ]:
### EMMA THIS IS THE CELL TO MODIFY

# Get indices of unique geopandas dataframes in case you selected the same glacier twice
unique_values, unique_indices = np.unique(np.array([i['id'] for i in gdf_list]), return_index=True)

# Iterate through the unique indices to recreate the list of geopandas dataframes unique only
gdf_list = [gdf_list[i] for i in unique_indices]

print(f"You have {len(gdf_list)} glaciers selected")

if len(urls) == 0:
    print('Select a datacube to fetch the data !!')

# Print the first dataframe to show the structure of it
gdf_list[0]

In [40]:
coords

[[29.276159989936716, 83.78609332964868]]

In [43]:
gpdf[0][0]

,id,CENLON,ZMAX,BGNDATE,ZMIN,RGIID,ASPECT,CENLAT,SLOPE,ZMED,...,TERMTYPE,O2REGION,STATUS,ENDDATE,FORM,SURGING,GLIMSID,O1REGION,NAME,geometry
0,RGI_CentralAsia.26883,83.787,6074,20090130,5665,RGI60-13.26885,324,29.276,16.7,5843,...,0,8,0,-9999999,0,9,G083787E29276N,13,CN5O257C0025,"MULTIPOLYGON (((83.78740 29.27100, 83.78710 29..."


In [44]:
urls

['http://its-live-data.s3.amazonaws.com/datacubes/v2/N20E080/ITS_LIVE_vel_EPSG32644_G0120_X750000_Y3250000.zarr']

In [14]:
#|export
try: 
    gpdf_ls = gpdf[0]
except:
    pass

In [16]:
#| export
def make_input_dict(coords, gpdf, urls):
    
    mod_urls = [re.sub(r'http', 's3', url) for url in urls]
    mod_urls = [re.sub(r'\.s3\.amazonaws\.com', '', url) for url in mod_urls]
    
    d = {'coords': coords,
     'gpdf': gpdf,
     'urls': mod_urls
        }
    return d

In [17]:
#| export
try:
    d = make_input_dict(coords, gpdf, URLs)
    urls = d['urls']
except:
    print(' no input data specified, have you made a map selection?')

 no input data specified, have you made a map selection?


In [18]:
#| export
def create_data_dict(urls):
    # Modify the urls so they can be opened by zarr (replace 'http' by 's3' and delete '.s3.amazonaws.com')
    mod_urls = [re.sub(r'http', 's3', url) for url in urls]
    mod_urls = [re.sub(r'\.s3\.amazonaws\.com', '', url) for url in urls]
    
    # Create storing arrays for the coordinates on-glacier
    X_valid = []
    Y_valid = []
    X_tot = []
    Y_tot = []
    
    # Create an empty directoryimport pickle to hold many variables all tied to the datacubes
    data_dict = {}
    

    # We iterate through the different datacubes so they can each have one instance of the variables below
    for mod_urls in urls:
        zarr_store = None # To store the datacube's information and access its variables
        dates = None # To store the dates at which the inversion will give values
        A_m = None # 1st part of the design matrix
        reg_mat_Inv = None # Regularization in time, 2nd part of the design matrix
        mission = None # If you want to invert specifically for one mission in particular ('S1','L8','L9', etc...)
        index_sort = None # Indices representing the sorted dates (from older to most recent)
        inds_mission = None # Indices representing the sorted dates per mission chosen
        ind_tot = None # Indices representing the indices of the pixels on the GOI
        valid_idx = None # Easting and Northing values of the indices above
        proj_cube = None # Projection of the datacube
        mask_dates = None # Mask that filters out dates outside of desired date range
        
        # Create a dictionary entry for the URL with the desired subsets
        data_dict[mod_urls] = {
            'zarr_store': zarr_store,
            'dates_noinv': dates,
            'A_m': A_m,
            'reg_mat_Inv': reg_mat_Inv,
            'mission': mission,
            'index_sort': index_sort,
            'inds_mission': inds_mission,
            'dates': dates,
            'ind_tot': ind_tot,
            'valid_idx': valid_idx,
            'proj_cube': proj_cube,
            'mask_dates': mask_dates
        }
        
    return data_dict, X_valid, Y_valid, X_tot, Y_tot


In [19]:
#| export 
def get_extents(url, gpdf, X_tot, Y_tot, X_valid, Y_valid, data_dict):# mission, lamb, derivative, day_interval):
    
    #url = input_data_dict['urls'].iloc[0]
    
    # Open the zarr files
    fs = s3fs.S3FileSystem(anon=True)
    store = zarr.open(s3fs.S3Map(url, s3=fs))
   
    # Update the dictionnary
    data_dict[url]['zarr_store'] = store

    # Get the cube's projection
    proj_cube = store.attrs['projection']

    # Load X and Y of the dataset
    X = store['x'][:]
    Y = store['y'][:]

    # Store the arrays in the total list
    X_tot.append(X)
    Y_tot.append(Y)

    # Load dimensions
    shape_arr = store['v'].shape
    
    Xs, Ys = np.meshgrid(X, Y)
    points = np.array((Xs.flatten(), Ys.flatten())).T

    idx_valid = []
    
    for b in range(len(gpdf)):
        mpath = mplp.Path(list(gpdf[b]['geometry'].to_crs(np.int(proj_cube)).boundary.explode(index_parts = True).iloc[0].coords))
        glacier_mask = mpath.contains_points(points).reshape(Xs.shape)
        # Grab the indices of the points inside the glacier
        idx_valid.append(np.array(np.where(glacier_mask==True)))
        
    idx_valid = np.hstack(idx_valid)
    # Store the valid indices
    data_dict[url]['valid_idx'] = idx_valid
    
    # Store the cube projection
    data_dict[url]['proj_cube'] = proj_cube
    
    # Store the coordinates of the valid Xs and Ys
    X_valid.append([Xs[idx_valid[0][i], idx_valid[1][i]] for i in range(len(idx_valid[0]))])
    Y_valid.append([Ys[idx_valid[0][i], idx_valid[1][i]] for i in range(len(idx_valid[0]))])
    
    return X_tot, Y_tot, X_valid, Y_valid

In [20]:
#| export
def design_matrices(url, mission, lamb, derivative, day_interval, sdate, edate):

    # If you passed 'mission' as an argument, it grabs the appropriate values
    if mission:
        # Get the indices of the mission
        filt1 = np.where(data_dict[urls[url]]['zarr_store']['satellite_img1'][:] == mission)
        filt2 = np.where(data_dict[urls[url]]['zarr_store']['satellite_img2'][:] == mission)
        inds_mission = np.intersect1d(filt1[0],filt2[0])

        # Grab only the indices corresponding to the missions
        mid_dates = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]')[inds_mission]
        im1 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img1'][:], dtype='timedelta64[D]')[inds_mission]
        im2 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img2'][:], dtype='timedelta64[D]')[inds_mission]
    else:
        # If 'None' was passed as a mission argument, we grab all the available data.
        inds_mission = None
        mid_dates = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]')
        im1 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img1'][:], dtype='timedelta64[D]')
        im2 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img2'][:], dtype='timedelta64[D]')
    
    # Get some arrays
    index_sort = np.argsort(np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]'))
    mid_dates = mid_dates[index_sort]
    im1 = im1[index_sort]
    im2 = im2[index_sort]

    # If sdate is later than the first available date, we find its corresponding index
    try:
        sdate_ind = np.where(mid_dates >= sdate)[0][0]
    except:
        sdate_ind = 0
    
    # If edate is sooner than the last available date, we find its corresponding index
    try:
        edate_ind = np.where(mid_dates > edate)[0][0]
    except:
        edate_ind = None
    
    # Create a False/True mask where True if the date is in the desired range
    mask_dates = np.full(mid_dates.shape, False)
    mask_dates[sdate_ind:edate_ind] = True

    # Keep only the values within the desired range
    mid_dates = mid_dates[mask_dates]
    im1 = im1[mask_dates]
    im2 = im2[mask_dates]

    # Check which im is the smallest (first image, it changes depending on ITS_LIVE's version)
    if im2[0] < im1[0]:
        temp = im1
        im1 = im2
        im2 = temp

    # Create the date array with the new interval dates
    dates_nonum = np.arange(mid_dates[0], mid_dates[-1], timedelta(days=day_interval)).astype(np.datetime64)

    # Convert to numerical
    dates = (dates_nonum - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    dt_start = (im1 - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    dt_end = (im2 - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')

    # --------------- DESIGN MATRICES --------------- 

    # Initialize matrix
    A_m = np.zeros((mid_dates.shape[0],dates.shape[0]))

    # We have to iterate through the satellite pairs that actually gave a measurement
    for j in range(1, len(mid_dates)):
    # current contents of your for loop

        # Find the middate that is the closest to dt_start (supequal)
        start = np.argmin(np.abs(dates-dt_start[j]))

        # Find the middate that is closest to dt_end (infequal)
        end = np.argmin(dt_end[j] - dates[dates <= dt_end[j]])

        # Divide 1 by the amount of middates between d_start and d_end 
        if end == A_m.shape[1]-1: # If the mid_date is at the end of the array (acquisition im2 equals last mid_date)
            A_m[j, start:] = 1/(1+A_m.shape[1]-start)
        else: # If the measurement is in A's bounds temporally (we can have a satellite pair with the 2nd pair being outside of our mid_dates)
            A_m[j, start:end+1] = 1/(1+end-start) # Attribute to each pixel in the timescale of the satellite pair, the 1/amount of pixel in the pairmid_dates.shape


    # Initialize regularization matrix
    if derivative == 1:
        reg_mat_Inv = np.zeros((A_m.shape[1] -1, A_m.shape[1]))

        for j in range(A_m.shape[1] -1):
            reg_mat_Inv[j, j] = -lamb/day_interval
            reg_mat_Inv[j, j+1] = lamb/day_interval

    elif derivative == 2:
        # Initialize 2nd derivative regularization matrix
        reg_mat_Inv = np.zeros((A_m.shape[1] -1, A_m.shape[1]))

        for j in range(A_m.shape[1] -2):
            reg_mat_Inv[j, j] = lamb/(day_interval**2)
            reg_mat_Inv[j, j+1] = -2*lamb/(day_interval**2)
            reg_mat_Inv[j, j+2] = lamb/(day_interval**2)
            
    data_dict[urls[url]]['A_m'] = A_m
    data_dict[urls[url]]['reg_mat_Inv'] = reg_mat_Inv
    data_dict[urls[url]]['mission'] = mission
    data_dict[urls[url]]['index_sort'] = index_sort
    data_dict[urls[url]]['inds_mission'] = inds_mission
    data_dict[urls[url]]['dates'] = dates_nonum
    data_dict[urls[url]]['dates_noinv'] = mid_dates
    data_dict[urls[url]]['mask_dates']= mask_dates
            
    return data_dict

In [21]:
#| export
def Inv_reg(vObs, data, fillvalue):
    
    # Grab observed velocities
    vObs = vObs[data['index_sort']]
    vObs = vObs[data['mask_dates']]
    
    # Filter out the missions we don't want
    if mission:
        vObs = vObs[data['inds_mission']]  
    
    # Mask the NaNs so we don't compute the inversion for empty rows
    mask = np.logical_not(np.equal(vObs, fillvalue))
    
    # Create a masked velocity vector
    vObs_masked = vObs[mask]
    
    # Append regularization terms to dObs
    vObs_masked= np.hstack((vObs_masked, np.zeros((data['reg_mat_Inv'].shape[0]))))
     
    # Assemble the design matrix
    A_des = np.vstack((data['A_m'][mask], data['reg_mat_Inv']))
    
    # Invert the velocities
    vInv = np.linalg.solve(A_des.T@A_des,A_des.T@vObs_masked)
    #vInv = scipy.linalg.solve(A_des.T@A_des,A_des.T@vObs_masked, lower = False, check_finite=False, assume_a='gen')
    
    return vInv


In [37]:
#| export 
try:
    data_dict, X_valid, Y_valid, X_tot, Y_tot = create_data_dict(urls)

    for url in tqdm(range(len(urls))):
        X_tot, Y_tot, X_valid, Y_valid = get_extents(urls[url], gpdf_ls, X_tot, Y_tot, X_valid, Y_valid, data_dict)

    mission = None # 'None' if you want all the data, 'S1' for Sentinel-1 only, 'L' for Landsat only, etc.. .
    lamb = 10 # Smoothing coefficient: the higher the value, the more the inversion favors a smooth output. BAD for surging glaciers, GOOD for non-surging glaciers
    derivative = 2 # Derivative degree for the inversion. Doesn't change much unless you have a specific reasong to choose 1 or 2 (1st or 2nd derivative)
    day_interval = 12 # Amount of days in between each inversion value. The higher, the faster the inversion. But you also lose in temporal resolution. 12 here because Sentinel-1 repeat-time is 12.
    sdate = None # Start date, format 'YYYY-MM-DD' OR None if you want to grab the entire timeseries available
    edate = None # End date, format 'YYYY-MM-DD' OR None if you want to grab the entire timeseries available
    GPU = True # None if you want to run it on CPU

    # Create Eastings and Northings arrays based on the Eastings and Northings of the datacubes
    X_arr = np.unique(np.hstack(X_tot))
    Y_arr = np.unique(np.hstack(Y_tot))
    
    # Crop to the GOI (so we avoid over-filling our matrix with NaNs)
    x_min = np.where(np.min(np.hstack(X_valid)) == X_arr)[0][0]
    x_max = np.where(np.max(np.hstack(X_valid)) == X_arr)[0][0]
    y_min = np.where(np.min(np.hstack(Y_valid)) == Y_arr)[0][0]
    y_max = np.where(np.max(np.hstack(Y_valid)) == Y_arr)[0][0]
    
    
    # And now search the indices corresponding to the coordinates 
    x_matches = np.hstack([[np.where(i == X_arr[min(x_min-1, x_max+1):max(x_min-1, x_max+1)])[0][0] for i in row] for row in X_valid]).astype(int)
    y_matches = np.hstack([[np.where(i == Y_arr[min(y_min-1, y_max+1):max(y_min-1, y_max+1)])[0][0] for i in row] for row in Y_valid]).astype(int)
    
    # Create an array representing the glacier
    template = np.zeros((len(Y_arr[min(y_min-1, y_max+1):max(y_min-1, y_max+1)]), len( X_arr[min(x_min-1, x_max+1):max(x_min-1, x_max+1)])))
    template[y_matches, x_matches] = 1
except:
    pass

0it [00:00, ?it/s]


In [28]:
try:
    plt.pcolormesh(X_arr[x_min-1:x_max+1], Y_arr[y_min-1:y_max+1], template)
except:
    pass

In [29]:
#| export
try:

    for i in tqdm(range(len(urls))):
        design_matrices(i, mission, lamb, derivative, day_interval)
except:
    pass

0it [00:00, ?it/s]


### Gather the dates for each datacube. They might not always correspond.

In [30]:
#| export
try:
    # Get the total amount of temporal steps
    ind_tot = []
    for i in urls:
        ind_tot.append(data_dict[i]['dates'])
    
    ind_tot = np.unique(np.hstack(ind_tot))
    
    for i in urls:
        data_dict[i]['ind_tot'] = np.array([np.where(c == ind_tot)[0][0] for c in data_dict[urls[0]]['dates']])
except:
    pass

### Calculate the point-inversion for all the GOI pixels in the datacubes for Vx and Vy.

In [31]:
#| export
if GPU:

    import torch

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    '''
    # Migrate to torch & GPU
    for c in range(len(urls)):
        data_dict[urls[c]]['A_m'] = torch.from_numpy(data_dict[urls[c]]['A_m']).to(device)
        data_dict[urls[c]]['reg_mat_Inv'] = torch.from_numpy(data_dict[urls[c]]['reg_mat_Inv']).to(device)
    '''
    
    def Inv_reg_torch(vObs, data, fillvalue, device):
     
        # Grab observed velocities
        vObs = vObs[data['index_sort']]
        vObs = vObs[data['mask_dates']]
        
        # Filter out the missions we don't want
        if mission:
            vObs = vObs[data['inds_mission']]  
        
        # Mask the NaNs so we don't compute the inversion for empty rows
        mask = np.logical_not(np.equal(vObs, fillvalue))
        
        # Create a masked velocity vector
        vObs_masked = torch.from_numpy(vObs[mask])
        
        # Append regularization terms to dObs
        vObs_masked= torch.hstack((vObs_masked, torch.zeros((data['reg_mat_Inv'].shape[0])))).to(device)
         
        # Assemble the design matrix
        A_des = torch.vstack((data['A_m'][mask], data['reg_mat_Inv']))
        
        # Invert the velocities
        vInv = torch.linalg.solve(A_des.T@A_des,A_des.T@vObs_masked.double())
        
        return vInv
    
    vxInv = torch.zeros((len(ind_tot), template.shape[0], template.shape[1])).double().to(device)
    vyInv = torch.zeros((vxInv.shape)).double().to(device)
    
    # Define the total number of iterations
    total_iterations = len(y_matches)
    
    # Create a tqdm object with dynamic_ncols=False to suppress intermediate updates
    # Create a tqdm object with a larger mininterval to suppress intermediate updates
    progress_bar = tqdm(total=total_iterations, dynamic_ncols=False, mininterval=1.0)
    
    
    i = 0
    for c in range(len(urls)):
        valid_idx = data_dict[urls[c]]['valid_idx']
        fillvx = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][0], valid_idx[1][0]].min() #data_dict[urls[c]]['zarr_store']['vx'].fill_value   fill_value is wrong in the new ITS_LIVE version
        fillvy = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][0], valid_idx[1][0]].min() #data_dict[urls[c]]['zarr_store']['vy'].fill_value
    
        for V in range(len(valid_idx[0])):
            vxObs = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][V], valid_idx[1][V]]
            vyObs = data_dict[urls[c]]['zarr_store']['vy'][:, valid_idx[0][V], valid_idx[1][V]]
            vxInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg_torch(vxObs, data_dict[urls[c]], fillvx, device)
            vyInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg_torch(vyObs, data_dict[urls[c]], fillvy, device)
            
            i += 1
            progress_bar.update(1)  # Update the progress bar

            if i%100 == 0:
                with open("Counter.txt", "w") as text_file:
                    text_file.write(f"Counter: {i}")
            
            if i%5000 == 0:
                
                print(f"Saved at {i}")
                # Get the names of all the glaciers in the datacube 
                #glac_names = np.hstack(np.array([glacier.id.values[0] for glacier in gdf_list]))
                #glac_names = '-'.join(glac_names)

                # Gather the projection of the cube
                #glac_proj = str(np.unique(np.hstack([data_dict[urls[i]]['proj_cube'] for i in range(len(urls))]))[0])

                # Create a new dataset with vx and vy, using attributes from 'ds'
                new_ds = xr.Dataset(
                    {
                        "vx": (["time", "y", "x"], vxInv.cpu().numpy()),
                        "vy": (["time", "y", "x"], vyInv.cpu().numpy()),
                    },
                    coords={
                        "time": ind_tot,
                        "x": X_arr[x_min-1:x_max+1],
                        "y": Y_arr[y_min-1:y_max+1],
                    },
                    attrs=data_dict[urls[0]]['zarr_store'].attrs,
                ).chunk({'time': 1, 'x': 100, 'y': 100})

                from dask.diagnostics import ProgressBar
                write_job = new_ds.to_netcdf(f'Inverted_Cube.nc', compute=False)
                with ProgressBar():
                    print(f"Writing to {'Inverted_Cube.nc'}")
                    write_job.compute()
    
    # Close the progress bar
    progress_bar.close()

    # Save the dataset
    new_ds = xr.Dataset(
        {
            "vx": (["time", "y", "x"], vxInv.cpu().numpy()),
            "vy": (["time", "y", "x"], vyInv.cpu().numpy()),
        },
        coords={
            "time": ind_tot,
            "x": X_arr[x_min-1:x_max+1],
            "y": Y_arr[y_min-1:y_max+1],
        },
        attrs=data_dict[urls[0]]['zarr_store'].attrs,
    ).chunk({'time': 1, 'x': 100, 'y': 100})
    
    from dask.diagnostics import ProgressBar
    write_job = new_ds.to_netcdf(f'VelInv.nc', compute=False)
    with ProgressBar():
        print(f"Writing to {'VelInv.nc'}")
        write_job.compute()



else:
    
    vxInv = np.zeros((len(ind_tot), template.shape[0], template.shape[1]))
    vyInv = np.zeros((vxInv.shape))
    
    # Define the total number of iterations
    total_iterations = len(y_matches)
    
    # Create a tqdm object with dynamic_ncols=False to suppress intermediate updates
    # Create a tqdm object with a larger mininterval to suppress intermediate updates
    progress_bar = tqdm(total=total_iterations, dynamic_ncols=False, mininterval=1.0)
    
    
    i = 0
    for c in range(len(urls)):
        valid_idx = data_dict[urls[c]]['valid_idx']
        fillvx = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][0], valid_idx[1][0]].min() #data_dict[urls[c]]['zarr_store']['vx'].fill_value   fill_value is wrong in the new ITS_LIVE version
        fillvy = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][0], valid_idx[1][0]].min() #data_dict[urls[c]]['zarr_store']['vy'].fill_value
        
        for V in range(len(valid_idx[0])):
            vxObs = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][V], valid_idx[1][V]]
            vyObs = data_dict[urls[c]]['zarr_store']['vy'][:, valid_idx[0][V], valid_idx[1][V]]
            vxInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg(vxObs, data_dict[urls[c]], fillvx)
            vyInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg(vyObs, data_dict[urls[c]], fillvy)
            
            i += 1
            if i%100 == 0:
                with open("Counter.txt", "w") as text_file:
                    text_file.write(f"Counter: {i}")
            
            if i%10000 == 0:
                
                print(f"Saved at {i}")
                # Get the names of all the glaciers in the datacube 
                #glac_names = np.hstack(np.array([glacier.id.values[0] for glacier in gdf_list]))
                #glac_names = '-'.join(glac_names)

                # Gather the projection of the cube
                #glac_proj = str(np.unique(np.hstack([data_dict[urls[i]]['proj_cube'] for i in range(len(urls))]))[0])

                # Create a new dataset with vx and vy, using attributes from 'ds'
                new_ds = xr.Dataset(
                    {
                        "vx": (["time", "y", "x"], vxInv),
                        "vy": (["time", "y", "x"], vyInv),
                    },
                    coords={
                        "time": ind_tot,
                        "x": X_arr[x_min-1:x_max+1],
                        "y": Y_arr[y_min-1:y_max+1],
                    },
                    attrs=data_dict[urls[0]]['zarr_store'].attrs,
                ).chunk({'time': 1, 'x': 100, 'y': 100})

                from dask.diagnostics import ProgressBar
                write_job = new_ds.to_netcdf(f'Inverted_Cube.nc', compute=False)
                with ProgressBar():
                    print(f"Writing to {'Inverted_Cube.nc'}")
                    write_job.compute()
            progress_bar.update(1)  # Update the progress bar
    
    # Close the progress bar
    progress_bar.close()

    # Save the dataset
    new_ds = xr.Dataset(
        {
            "vx": (["time", "y", "x"], vxInv),
            "vy": (["time", "y", "x"], vyInv),
        },
        coords={
            "time": ind_tot,
            "x": X_arr[x_min-1:x_max+1],
            "y": Y_arr[y_min-1:y_max+1],
        },
        attrs=data_dict[urls[0]]['zarr_store'].attrs,
    ).chunk({'time': 1, 'x': 100, 'y': 100})

    from dask.diagnostics import ProgressBar
    write_job = new_ds.to_netcdf(f'Inverted_Cube.nc', compute=False)
    with ProgressBar():
        print(f"Writing to {'Inverted_Cube.nc'}")
        write_job.compute()

### Save the results in a netcdf file

In [32]:
#| export
# Create a new dataset with vx and vy, using attributes from 'ds'
try:
    new_ds = xr.Dataset(
        {
            "vx": (["time", "y", "x"], vxInv),
            "vy": (["time", "y", "x"], vyInv),
        },
        coords={
            "time": ind_tot,
            "x": X_arr[x_min-1:x_max+1],
            "y": Y_arr[y_min-1:y_max+1],
        },
        attrs=data_dict[urls[0]]['zarr_store'].attrs,
    ).chunk({'time': 1, 'x': 100, 'y': 100})
    
    from dask.diagnostics import ProgressBar
    write_job = new_ds.to_netcdf('Inverted_Cube.nc', compute=False)
    with ProgressBar():
        print(f"Writing to {'Inverted_Cube.nc'}")
        write_job.compute()
except:
    pass

In [33]:
try: 
    test = xr.open_dataset('Inverted_Cube.nc')
except:
    pass